![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

Your task is to first identify the three best-performing industries based on the number of new unicorns created in 2019, 2020, and 2021 combined.

In [36]:
SELECT 
    industries.industry, 
    COUNT(*) AS unicorn_count
FROM 
    public.industries
LEFT JOIN 
    public.dates ON dates.company_id = industries.company_id
WHERE 
    EXTRACT(YEAR FROM dates.date_joined) IN (2019, 2020, 2021)
GROUP BY 
    industries.industry
ORDER BY 
    unicorn_count DESC
LIMIT 3;

,industry,unicorn_count
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


From those industries (1), you will need to find the number of unicorns within these industries (2), the year that they became a unicorn (3), and their average valuation, converted to billions of dollars and rounded to two decimal places (4).

In [37]:
SELECT
    industries.industry,
    COUNT(*) AS unicorns_in_industry,
    EXTRACT(YEAR FROM dates.date_joined) AS year_joined,
    ROUND(AVG(funding.valuation) / 1000000000, 2) AS avg_valuation_in_billions
FROM industries
LEFT JOIN dates
ON dates.company_id = industries.company_id
LEFT JOIN funding
ON funding.company_id = industries.company_id
WHERE industries.industry IN ('Fintech', 'Internet software & services', 'E-commerce & direct-to-consumer') 
	AND 
	EXTRACT(YEAR FROM dates.date_joined) IN (2019, 2020, 2021)
GROUP BY
    industries.industry, 
    EXTRACT(YEAR FROM dates.date_joined);

,industry,unicorns_in_industry,year_joined,avg_valuation_in_billions
0,E-commerce & direct-to-consumer,12,2019,2.58
1,E-commerce & direct-to-consumer,16,2020,4.00
2,E-commerce & direct-to-consumer,47,2021,2.47
3,Fintech,20,2019,6.80
4,Fintech,15,2020,4.33
5,Fintech,138,2021,2.75
6,Internet software & services,13,2019,4.23
7,Internet software & services,20,2020,4.35
8,Internet software & services,119,2021,2.15


With the above information you can then finish your query to return a table containing industry, year, num_unicorns, and average_valuation_billions. For readability, the firm have asked you to sort your results by year and number of unicorns, both in descending order.

In [38]:
SELECT
    industries.industry,
	EXTRACT(YEAR FROM dates.date_joined) AS year,
    COUNT(*) AS num_unicorns,
    ROUND(AVG(funding.valuation) / 1000000000, 2) AS average_valuation_billions
FROM industries
LEFT JOIN dates
ON dates.company_id = industries.company_id
LEFT JOIN funding
ON funding.company_id = industries.company_id
WHERE industries.industry IN ('Fintech', 'Internet software & services', 'E-commerce & direct-to-consumer') 
	AND 
	EXTRACT(YEAR FROM dates.date_joined) IN (2019, 2020, 2021)
GROUP BY
    industries.industry, 
    EXTRACT(YEAR FROM dates.date_joined)
ORDER BY 
    year DESC, 
    num_unicorns DESC;

,industry,unicorns_in_industry,year_joined,avg_valuation_in_billions
0,Fintech,138,2021,2.75
1,Internet software & services,119,2021,2.15
2,E-commerce & direct-to-consumer,47,2021,2.47
3,Internet software & services,20,2020,4.35
4,E-commerce & direct-to-consumer,16,2020,4.00
5,Fintech,15,2020,4.33
6,Fintech,20,2019,6.80
7,Internet software & services,13,2019,4.23
8,E-commerce & direct-to-consumer,12,2019,2.58
